In [1]:
import sys
import os
import importlib

import pandas as pd
import seaborn as sns

import dart.Util
import dart.metrics.start_calculations

In [ ]:
config = {
    'test_size': 1000, 
    'metrics':  ['calibration', 'fragmentation', 'activation', 'representation', 'alternative_voices'],
    'cutoff': [1, 2, 5, 10, 20, 0],
    'algorithms': ['lstur', 'pop', 'random'],
    'output_folder': 'output/',
    'language': 'english'
}

articles, recommendations, behavior_file = dart.Util.read_files()

In [ ]:
calculator = dart.metrics.start_calculations.MetricsCalculator(config, articles, recommendations, behavior_file)

In [ ]:
df = calculator.execute()

In [ ]:
articles

In [ ]:
recommendations.loc['59012']